In [1]:
from IPython.display import display
import numpy as np
# import modin.pandas as pd
import pandas as pd
import datetime
import time
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE, f_regression
from sklearn.metrics import r2_score

import itertools

import warnings
import json

warnings.filterwarnings('ignore')

plt.style.use("fivethirtyeight")
sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

# np.random.seed(4590)
df = pd.read_csv(r'../../../Data/train_data.csv')
df.head(5)


,企业编号,企业总评分,软著数量,作品著作数量,项目数量,纳税A级年份_2014,纳税A级年份_2015,纳税A级年份_2016,纳税A级年份_2017,纳税A级年份增长1,...,应收账款周转天数(天)_mean,应收账款周转天数(天)_max,应收账款周转天数(天)_min,应收账款周转天数(天)_std,应收账款周转天数(天)滚动增长_mean,存货周转天数(天)_mean,存货周转天数(天)_max,存货周转天数(天)_min,存货周转天数(天)_std,存货周转天数(天)滚动增长_mean
0,1001,75.374276,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,107.589270,191.707773,63.791689,44.495607,0.151392,414.778035,1089.655763,176.283983,325.371499,1.562757
1,1002,79.830122,2.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,...,46.903333,56.590000,39.830000,6.234116,0.023916,6.506667,7.040000,5.010000,0.702335,0.045330
2,1003,78.318264,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,84.275556,139.910000,56.020000,33.143654,-0.040224,54.918889,75.540000,38.010000,11.089465,0.031792
3,1004,83.253376,0.0,6.0,1.0,0.0,0.0,2.0,1.0,0.0,...,26.720000,35.360000,17.290000,6.024438,0.081857,6.954444,7.900000,6.240000,0.618448,0.021711
4,1005,83.291493,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,94.050000,110.260000,77.850000,9.652235,0.012921,108.584444,357.190000,44.160000,101.728838,0.344086


In [2]:
y = df[['企业编号', '企业总评分']]
x = df.drop(['企业总评分'], axis=1)

xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=31)
ytrain_id = ytrain['企业编号']
ytrain = ytrain['企业总评分']
ytest_id = ytest['企业编号']
ytest = ytest['企业总评分']
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

id_train = xtrain['企业编号']
id_test = xtest['企业编号']
xtrain.drop(['企业编号'], axis=1, inplace=True)
xtest.drop(['企业编号'], axis=1, inplace=True)
feature_name=xtrain.columns.values

(2364, 335) (592, 335) (2364,) (592,)


# RFE

In [3]:
estimator = lgb.LGBMRegressor(objective='regression',
                              num_leaves=20,
                              learning_rate=0.005,
                              n_estimators=1500,
                              max_depth=7,
                              boosting='gbdt',
                              metric='rmse',
                              max_bin=35,
                              n_jobs=-1,
                              min_child_samples=100,
                              bagging_seed=11,
                              bagging_fraction=0.7,
                              bagging_freq=1,
                              min_data_in_leaf=20
                             )

In [13]:
for i in range(334,120, -5):
    selector = RFE(estimator, i, step=1)
    selector = selector.fit(xtrain, ytrain)
    pre=selector.predict(xtest)
    print(i, np.sqrt(mean_squared_error(pre,ytest)))
# The best is 124

269 3.0638521283170714
264 3.0616603110561837
259 3.0582776787114994
254 3.061953415855858
249 3.064838436713015
244 3.0615766009893934
239 3.060279022738049
234 3.0600450908996812
229 3.059858904321477
224 3.061015185205892
219 3.0615174770186724
214 3.061374247166835
209 3.0645782046830456
204 3.0575176340169508
199 3.05967670674765
194 3.0575021421427078
189 3.0574910247895892
184 3.058271958504811
179 3.059158845335394
174 3.0589430725151074
169 3.059898782543876
164 3.058495131038808
159 3.0610546350674945
154 3.0636838111748896
149 3.062313030109125
144 3.057065568319598
139 3.0610593012179006
134 3.060521397964574
129 3.061921885299704
124 3.054093593848125


In [4]:
selector = RFE(estimator, 124, step=1)
selector = selector.fit(xtrain, ytrain)
pre=selector.predict(xtest)
print(np.sqrt(mean_squared_error(pre,ytest)), r2_score(pre,ytest))

NameError: name 'i' is not defined

# Test data

In [7]:
test_df = pd.read_csv(r'../../../Data/test_data.csv')
X_test = test_df.drop(['企业编号'], axis=1)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test = X_test.fillna(0)
test_id = test_df['企业编号']

In [8]:
pre=selector.predict(X_test)

In [10]:
result=pd.DataFrame({
    '企业编号':test_id,
    'pre_rating':pd.Series(pre).astype(int)
})

result.to_excel(r'./赛题1结果_Trainee.xlsx', index=False,header=False)